In [1]:
#!pip install hanja
import torch
from torch import nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
%matplotlib inline
import datetime as dt
import itertools
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import codecs
import re
import logging
import datetime
from torch import autograd, nn
import re
import os
from hanja import hangul
import operator
from random import shuffle;
import time;
import math

In [2]:
# from google.colab import drive
# drive.mount('/content/gdirve/')

In [3]:
path = '/Users/soankim/Desktop/Experiment/2019_MPI/Simulation/'
os.chdir(path)
file = 'pyojun_pron.xlsx'
file = pd.read_excel(file)
file.tail() # shape: 427353, 3

,index,word,pron
427348,NaN,크로뮴산암모늄,크로뮴산암모늄
427349,NaN,크로뮴산연,크로뮴산연
427350,NaN,크로뮴산칼륨,크로뮴산칼륨
427351,NaN,크로뮴염료,크로뮴염료
427352,NaN,크로뮴옐,크로뮴옐


In [4]:
twos = file[file['word'].str.len()==2]
twos.shape

(84703, 3)

In [5]:
valid_idx = file.dropna('index')
valid_idx['index'] = valid_idx['index'].astype(int)
valid_idx.head() # 명사: SP_CODE ==1, 동사: SP_CODE == 5

/Users/soankim/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,index,word,pron
0,6529,갈아입다,갈아입따
1,5520,각심,각씸
2,5522,각심하다,각씸하다
3,5525,각심소원,각씸소원
4,5526,각심이,각씨미


In [6]:
pron = pd.read_csv('/Users/soankim/Desktop/Experiment/표준국어대사전/20180920_table_csv/사전데이터_csv/필요파일/WORDS_CR_DATA_TABLE.csv')
#pron.loc[pron['WORD_NO']==6529] # 갈아입다
pron.loc[pron['WORD_NO']==5520]

,WORD_NO,WORD,SUP_NO,WORD_SEQ,ORG_WORD,WORD_TP_CODE,WORD_KIND_CODE,UPPER_WORD_NO,OWN_YN,CHN_YN,FRN_YN,ORG_LANGUAGE,ORIGIN,LETTER,ID,SOUND_SEQ,CREATOR,CREATED_TIME,UPDATOR,UPDATED_TIME
9149,5520,각심,1,5925000,각심,1,0,NaN,1,0,0,NaN,NaN,ㄱ,527300.0,NaN,SYSTEM,07/04/09,NaN,NaN


In [7]:
part_sp = pd.read_csv('/Users/soankim/Desktop/Experiment/표준국어대사전/20180920_table_csv/사전데이터_csv/필요파일/SPEECHPART_CR_DATA_TABLE.csv')
part_sp.loc[part_sp['WORD_NO']==5520]

/Users/soankim/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,WORD_NO,SP_NO,SP_SEQ,SP_CODE,WORD_GRP_CODE,SPELLING_RULE,TECH_TERM_CODE,PATTERN,GRAMMAR,PATTERN_ETC,CREATOR,CREATED_TIME,UPDATOR,UPDATED_TIME
3449,5520,9953,0,1,0,NaN,320,NaN,NaN,NaN,SYSTEM,07/04/10,NaN,NaN


In [8]:
dic = part_sp.set_index('WORD_NO').transpose().to_dict(orient='list')

/Users/soankim/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  """Entry point for launching an IPython kernel.


In [9]:
pos = []
for idx in valid_idx['index']:
    pos.append(dic[idx][2])

In [10]:
valid_idx['part'] = pos
valid_idx.tail()

/Users/soankim/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,index,word,pron,part
92319,309710,진탕증,진탕쯩,1
92320,309718,지엣굴,지엗꿀,1
92321,310161,주작부언,주작뿌언,1
92322,310162,주작부언하다,주작뿌언하다,5
92323,309735,지역적,지역쩍,15


In [11]:
nouns = valid_idx.loc[valid_idx['part']==1]
nouns.shape

(67131, 4)

In [12]:
ones = nouns.loc[nouns['word'].str.len()==1]
ones.shape

(264, 4)

In [13]:
ones.head()

,index,word,pron,part
88,5780,감,감,1
566,19032,겻,겯,1
1237,20349,계,계,1
1348,7686,갈,갈,1
1488,28833,곬,골,1


In [14]:
twos = nouns.loc[nouns['word'].str.len()==2]
twos.shape

(21877, 4)

In [15]:
twos.head()

,index,word,pron,part
1,5520,각심,각씸,1
8,5534,간문,간문,1
9,5655,갈수,갈쑤,1
16,5984,간경,간경,1
18,6843,감심,감심,1


In [16]:
threes = nouns.loc[nouns['word'].str.len()==3]
threes.shape

(29002, 4)

In [17]:
threes.head()

,index,word,pron,part
4,5526,각심이,각씨미,1
5,5530,간무봉,간무봉,1
10,5656,갈수년,갈쑤년,1
11,5658,감각력,감강녁,1
12,5659,감각령,감강녕,1


In [18]:
fours = nouns.loc[nouns['word'].str.len()==4]
fours.shape

(12453, 4)

In [19]:
fours.head()

,index,word,pron,part
3,5525,각심소원,각씸소원,1
25,6851,감소수열,감소수열,1
29,5416,간접떼기,간접떼기,1
32,5760,간소살사,간소살싸,1
33,5761,간소포획,간소포획,1


## Start

In [20]:
fir_o = []
sec_o = []
for syl in twos['word']:
    fir_o.append(hangul.separate(syl[0]))
    sec_o.append(hangul.separate(syl[1]))

fir_p = []
sec_p = []
for syl in twos['pron']:
    fir_p.append(hangul.separate(syl[0]))
    sec_p.append(hangul.separate(syl[1]))

In [21]:
temp_o1 = pd.DataFrame(fir_o)
temp_o2 = pd.DataFrame(sec_o)

temp_p1 = pd.DataFrame(fir_p)
temp_p2 = pd.DataFrame(sec_p)

df_o = pd.concat([temp_o1, temp_p2]).sort_index().reset_index(drop=True)
df_p = pd.concat([temp_o2, temp_o1]).sort_index().reset_index(drop=True) 
df=pd.concat([df_o, df_p]).sort_index().reset_index(drop=True)

In [22]:
df

,0,1,2
0,0,0,1
1,9,20,16
2,0,0,1
3,10,20,16
4,6,13,4
...,...,...,...
87503,14,0,21
87504,9,8,1
87505,12,20,8
87506,10,8,1


In [23]:
hangul.build(10, 8, 1)

'쏙'

In [24]:
o_syl = tuple(map(operator.add, fir_o, sec_o))
p_syl = tuple(map(operator.add, fir_p, sec_p))
df = pd.DataFrame({"ortho": o_syl, "phono": p_syl})
df_o = pd.DataFrame(df['ortho'].values.tolist(), index=df.index)  
df_p = pd.DataFrame(df['phono'].values.tolist(), index=df.index) 

# df=pd.concat([df_o, df_p]).sort_index().reset_index(drop=True)
# #df.columns = ['o1', 'o2', 'o3', 'o4', 'o5','o6', 'p1', 'p2', 'p3', 'p4', 'p5','p6']
# df.head()

In [25]:
df_o = pd.DataFrame.to_csv(df_o, 'df_o.csv', header=False, index=False)
df_p = pd.DataFrame.to_csv(df_p, 'df_p.csv', header=False, index=False)

In [26]:
# pd.DataFrame.to_csv(df, '/Users/soankim/Desktop/df.csv', header=False, index=False)
# df = pd.DataFrame.to_csv(df, '/Users/soankim/Desktop/df.csv', header=False, index=False)

In [27]:
df_o = np.loadtxt('df_o.csv', delimiter= ',') #could not convert string to
df_p = np.loadtxt('df_p.csv', delimiter= ',') #could not convert string tofloat: '19990104,1398.59'

In [28]:
# df = np.loadtxt('/Users/soankim/Desktop/df.csv', delimiter= ',')

In [29]:
df_o

array([[ 0.,  0.,  1.,  9., 20., 16.],
       [ 0.,  0.,  4.,  6., 13.,  4.],
       [ 0.,  0.,  8.,  9., 13.,  0.],
       ...,
       [12., 20., 17.,  0.,  6., 21.],
       [14.,  0., 21., 12.,  8.,  0.],
       [12., 20.,  8.,  9.,  8.,  1.]])

In [30]:
item_num = twos.shape[0]
o_slots = 68 * 2
p_slots = 68 * 2
hid_op = 100
lr = 0.00001

In [31]:
def binary_generator(word):
    binary_mat = np.zeros((1, 68*2));
    
    for syl_idx in range(len(word)):
        syllable = word[syl_idx]
        binary_idx = hangul.separate(syllable)
        binary_mat[0, syl_idx * 68 + binary_idx[0]] =1 # initial consonant
        binary_mat[0, syl_idx * 68 + 19 + binary_idx[1]] =1 # vowel
        binary_mat[0, syl_idx * 68 + 19 + 21 + binary_idx[2]] =1 # final consonant
    
    return binary_mat

In [63]:
word_list = twos['word'].to_list()
pron_list = twos['pron'].to_list()

# freq_list = np.random.rand(len(word_list)) # (1, len(word_list)) X

ortho_mat = np.zeros((item_num, o_slots))
for word_idx in range(len(word_list)):
    ortho_mat[word_idx] = binary_generator(word_list[word_idx])
    
phono_mat = np.zeros((item_num, p_slots))
for word_idx in range(len(pron_list)):
    phono_mat[word_idx] = binary_generator(pron_list[word_idx])

# freq_mat = np.zeros((item_num, 1))
# for freq_idx in range(len(freq_list)):
#     freq = freq_list[freq_idx]
#     freq_mat[freq_idx] = 0.02 * np.log2(freq + 2)
    
print("ortho_mat_size: ", ortho_mat.shape)
print("phono_mat_size: ", phono_mat.shape)
# print("freq_mat_size: ", freq_mat.shape)

ortho_mat_size:  (21877, 136)
phono_mat_size:  (21877, 136)


In [64]:
def normalize_rows(x: np.ndarray):
    return x/np.linalg.norm(x, ord=2, axis=1, keepdims=True)

In [65]:
weight_mat_o_op = np.random.rand(o_slots, hid_op)
## Initializing weight matrix
weight_mat_o_op = normalize_rows(weight_mat_o_op)*math.sqrt(o_slots/2)

weight_mat_op_p = np.random.rand(hid_op, p_slots) 
weight_mat_op_p = normalize_rows(weight_mat_op_p)*math.sqrt(o_slots/2)

bias_mat_op = np.zeros((1, hid_op))
bias_mat_p = np.zeros((1, p_slots))

print("weight matrices")
print("o -> op: ", weight_mat_o_op.shape)
print("op -> p: ", weight_mat_op_p.shape)
print("")
print("bias matrices")
print("op: ", bias_mat_op.shape)
print("phono: ", bias_mat_p.shape)

weight matrices
o -> op:  (136, 100)
op -> p:  (100, 136)

bias matrices
op:  (1, 100)
phono:  (1, 136)


In [59]:
def Tanh(x):
    return np.sinh(x) / np.cosh(x)
    
def Sigmoid(x):
    return 1 / (1 + np.exp(-1 * x));

def Softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis = 1, keepdims=True)

def ReLU(x):
    return np.maximum(0, x);

def MSE(target_Pattern, output_Activation):
    return np.sqrt(np.mean((target_Pattern - output_Activation) ** 2, axis = 1));

In [60]:
mse_li = []

for idx in range(item_num):
    o_op_act = Softmax(np.dot(ortho_mat[idx, :], weight_mat_o_op)+ bias_mat_op)
    
    op_p_act = Softmax(np.dot(o_op_act, weight_mat_op_p)+ bias_mat_p)
    phono_act = Softmax(op_p_act + bias_mat_p)
    
    mse = MSE(phono_mat[idx, :], phono_act)
    mse_li.append(mse)
print("MSE average :", np.mean(mse_li))

MSE average : 0.2086214885194756


In [66]:
iteration = 100000
batch_size = 100
train_ratio = 0.7
train_size = int(item_num*train_ratio)
train_set = twos[0:train_size]
test_size = int(1-train_size)
test_set = twos[train_size:-1]
start_time = time.time();
for epoch in range(iteration):
    freq_filter = np.random.rand(train_size, 1) #15313
    train_idx_li = list(range(train_set.shape[0]))
    shuffle(train_idx_li)
    mse_li = []
    for initial_idx in range(0, train_size, batch_size):
        train_idx = train_idx_li[initial_idx:initial_idx+batch_size]
        input_pat = ortho_mat[train_idx, :]
        target_pat = phono_mat[train_idx, :]
        current_freq_filter = freq_filter[train_idx, :]
        op_act = Sigmoid(np.dot(input_pat, weight_mat_o_op)+ bias_mat_op)
        phono_act = Sigmoid(np.dot(op_act, weight_mat_op_p) + bias_mat_p)
        mse = MSE(phono_mat[train_idx, :], phono_act)
        mse_li.extend(mse)
        phono_error = (target_pat - phono_act) * current_freq_filter
        op_error = np.dot(phono_error, np.transpose(weight_mat_op_p))*np.sign(op_act)*current_freq_filter
        weight_mat_o_op += lr * np.dot(np.transpose(input_pat), op_error)
        weight_mat_op_p += lr * np.dot(np.transpose(op_act), phono_error)
        bias_mat_op += lr * np.sum(op_error, axis=0, keepdims=True)
        bias_mat_p += lr * np.sum(phono_error, axis=0, keepdims = True) 
    if (epoch + 1) % int(iteration/1000)==0:
        print("epoch", epoch, "Average MSE :", np.mean(mse_li));
print("\n", "time spent: ", time.time()-start_time, "s", sep="")

epoch 99 Average MSE : 0.22434261446160558
epoch 199 Average MSE : 0.20178116455695852
epoch 299 Average MSE : 0.19709926768345892
epoch 399 Average MSE : 0.19552629149472214
epoch 499 Average MSE : 0.1948370282219185
epoch 599 Average MSE : 0.1944816190877491
epoch 699 Average MSE : 0.1942760737127382
epoch 799 Average MSE : 0.19414774468893442
epoch 899 Average MSE : 0.1940621205749889
epoch 999 Average MSE : 0.1940029166048654
epoch 1099 Average MSE : 0.19396014675812556
epoch 1199 Average MSE : 0.19392799339439892
epoch 1299 Average MSE : 0.19390396775058044
epoch 1399 Average MSE : 0.1938848941778395
epoch 1499 Average MSE : 0.19386998337129843
epoch 1599 Average MSE : 0.19385764487880605
epoch 1699 Average MSE : 0.19384766932854627
epoch 1799 Average MSE : 0.19383959775382573
epoch 1899 Average MSE : 0.19383321832907643
epoch 1999 Average MSE : 0.19382726467573302
epoch 2099 Average MSE : 0.19382226320145074
epoch 2199 Average MSE : 0.19381793327482272
epoch 2299 Average MSE : 0.

/Users/soankim/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in exp
  """


epoch 7399 Average MSE : 0.19378098010833064
epoch 7499 Average MSE : 0.19378078625083603
epoch 7599 Average MSE : 0.19378050781044817
epoch 7699 Average MSE : 0.19378040013972864
epoch 7799 Average MSE : 0.19378044964818691
epoch 7899 Average MSE : 0.1937803954926243
epoch 7999 Average MSE : 0.1937802753592575
epoch 8099 Average MSE : 0.19378027238160778
epoch 8199 Average MSE : 0.19378015599873025
epoch 8299 Average MSE : 0.19378005060018202
epoch 8399 Average MSE : 0.19377986212081194
epoch 8499 Average MSE : 0.19377969861974143
epoch 8599 Average MSE : 0.1937800623375252
epoch 8699 Average MSE : 0.1937799080347532
epoch 8799 Average MSE : 0.19377982502393673
epoch 8899 Average MSE : 0.19377965598013738
epoch 8999 Average MSE : 0.19377964296621272
epoch 9099 Average MSE : 0.19377970508032635
epoch 9199 Average MSE : 0.19377982167226812
epoch 9299 Average MSE : 0.1937794862799864
epoch 9399 Average MSE : 0.19377938803508937
epoch 9499 Average MSE : 0.19377945659195486
epoch 9599 Aver

In [ ]:
mse_li = []

for epoch in range(iteration):
    
    test_idx_li = list(range(train_set.shape[0],item_num)) #6564
    shuffle(test_idx_li)
    
    for test_idx in test_idx_li:
        
        input_pat = ortho_mat[test_idx, :]
        target_pat = phono_mat[test_idx, :]

        op_act = Sigmoid(np.dot(input_pat, weight_mat_o_op)+ bias_mat_op)
        phono_act = Sigmoid(np.dot(op_act, weight_mat_op_p) + bias_mat_p)

        mse = MSE(phono_mat[idx, :], phono_act)
        mse_li.extend(mse)
print("MSE average :", np.mean(mse_li))

/Users/soankim/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in exp
  """
